In [14]:
import scanpy as sc
import scipy.sparse
import pandas as pd
import anndata

In [37]:
# update the paths here with the 50+ files I shared with you
paths = [
    "/mnt/cbbi-data00/internal_data/20190401_001/CSEQ-000258/raw_feature_bc_matrix.h5",
    "/mnt/cbbi-data00/internal_data/20190401_001/CSEQ-000259/raw_feature_bc_matrix.h5"
]

In [33]:
var_names = None
gexs = []
obss = []
for i, path in enumerate(paths):
    sample_name = f"Sample{i+1:0>3}"
    
    cur_adata = sc.read_10x_h5(path)
    
    # this filter is currently missing in our version but will almost certainly speed up access to
    # the raw data just by adding these two lines of code
    cur_adata.obs["umi_counts"] = cur_adata.X.sum(axis=1).A.flatten()
    cur_adata = cur_adata[
        cur_adata.obs["umi_counts"]>0
    ]
    
    cur_adata.obs_names = sample_name + ":" + cur_adata.obs_names.str.rsplit("-").str[0]
    cur_adata.obs["sample"] = sample_name
    
    print(path, cur_adata.shape)
    
    gexs.append(cur_adata.X.tocsr())
    obss.append(cur_adata.obs)
    
    if var_names is None:
        var_names = cur_adata.var_names
    else:
        assert (var_names == cur_adata.var_names).all()

Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Trying to set attribute `.obs` of view, making a copy.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Trying to set attribute `.obs` of view, making a copy.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


/mnt/cbbi-data00/internal_data/20190401_001/CSEQ-000258/raw_feature_bc_matrix.h5 (256753, 33538)


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Trying to set attribute `.obs` of view, making a copy.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Trying to set attribute `.obs` of view, making a copy.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.


/mnt/cbbi-data00/internal_data/20190401_001/CSEQ-000259/raw_feature_bc_matrix.h5 (178846, 33538)


In [34]:
gex = scipy.sparse.vstack(gexs)

obs = pd.concat(obss)

adata = anndata.AnnData(gex)
adata.var = cur_adata.var
adata.var_names = var_names
adata.obs = obs

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


In [35]:
adata

AnnData object with n_obs × n_vars = 435599 × 33538 
    obs: 'umi_counts', 'sample'
    var: 'gene_ids', 'feature_types', 'genome'

In [36]:
adata.obs.sample(5)

,umi_counts,sample
Sample001:GAGGTGATCCTAGTGA,1.0,Sample001
Sample001:TTTCCTCGTGAGCGAT,2.0,Sample001
Sample001:TGAGCCGTCATGTGGT,9358.0,Sample001
Sample001:CCTTTCTTCAAACAAG,3998.0,Sample001
Sample001:CTCTACGCATGTCGAT,1.0,Sample001


## What we want

* We want to be able to easily create a new filtered_adata matrix, eg changing the umi_counts threshold from 1000 to 500.
* We want to have relatively easy access to the unfiltered data from each sample, primarily for qc purposes (ie, we don't really need to access all samples at once, we need to access each unfiltered sample separately but efficiently)

In [23]:
filtered_adata = adata[adata.obs.eval("umi_counts>1000")]
filtered_adata

View of AnnData object with n_obs × n_vars = 9860 × 33538 
    obs: 'umi_counts'
    var: 'gene_ids', 'feature_types', 'genome'